In [30]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import seaborn as sns
import time
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [5]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In [7]:
# alternative methods of loading keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [3]:
hdb_df = pd.read_csv("../datasets/final/combined.csv")

In [50]:
hdb_df.columns

Index(['storey_range', 'floor_area_sqm', 'remaining_lease', 'resale_price',
       'bedok', 'bishan', 'bukit_batok', 'bukit_merah', 'bukit_panjang',
       'bukit_timah', 'central_area', 'choa_chu_kang', 'clementi', 'geylang',
       'hougang', 'jurong_east', 'jurong_west', 'kallang_whampoa',
       'marine_parade', 'pasir_ris', 'punggol', 'queenstown', 'sembawang',
       'sengkang', 'serangoon', 'tampines', 'toa_payoh', 'woodlands', 'yishun',
       'model_adjoined_flat', 'model_apartment', 'model_dbss',
       'model_improved', 'model_improved_maisonette', 'model_maisonette',
       'model_model_a', 'model_model_a2', 'model_model_a_maisonette',
       'model_multi_generation', 'model_new_generation',
       'model_premium_apartment', 'model_premium_apartment_loft',
       'model_premium_maisonette', 'model_simplified', 'model_standard',
       'model_terrace', 'model_type_s1', 'model_type_s2', 'type_2_room',
       'type_3_room', 'type_4_room', 'type_5_room', 'type_executive',
     

In [15]:
X = hdb_df.drop(columns=['resale_price'])
y = hdb_df['resale_price']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=42)

In [53]:
X_train.shape

(64324, 80)

In [12]:
X_valid.shape

(16081, 80)

In [29]:
hdb_df.describe()

,storey_range,floor_area_sqm,remaining_lease,resale_price,bedok,bishan,bukit_batok,bukit_merah,bukit_panjang,bukit_timah,...,dist_hawker,dist_park,dist_after_death,dist_attraction,dist_pool,dist_gym,dist_tennis,dist_stadium,dist_sports_hall,dist_library
count,80405.000000,80405.000000,80405.000000,8.040500e+04,80405.000000,80405.000000,80405.000000,80405.000000,80405.000000,80405.000000,...,80405.000000,8.040500e+04,8.040500e+04,8.040500e+04,8.040500e+04,8.040500e+04,8.040500e+04,8.040500e+04,8.040500e+04,8.040500e+04
mean,7.430794,97.581557,74.015546,4.399488e+05,0.060058,0.019551,0.039077,0.038816,0.035296,0.002637,...,1153.316828,1.556909e+04,1.939036e+04,1.743459e+04,1.615173e+04,1.637717e+04,1.819368e+04,1.665706e+04,1.638795e+04,1.596172e+04
std,5.602465,24.242844,11.518088,1.457353e+05,0.237597,0.138452,0.193780,0.193157,0.184529,0.051281,...,1816.462424,4.123750e+05,4.122225e+05,4.122466e+05,4.123578e+05,4.123500e+05,4.122894e+05,4.123405e+05,4.123497e+05,4.124073e+05
min,1.000000,31.000000,47.000000,1.600000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.311000e+01,6.104000e+01,1.507000e+01,1.226000e+01,1.226000e+01,3.022000e+01,3.022000e+01,1.226000e+01,2.037000e+01
25%,4.000000,76.000000,66.000000,3.380000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,359.750000,4.812400e+02,2.196710e+03,1.473330e+03,8.153400e+02,9.404300e+02,1.608110e+03,1.000470e+03,9.502500e+02,7.358400e+02
50%,7.000000,96.000000,73.000000,4.090000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,763.220000,7.187000e+02,3.461760e+03,2.402940e+03,1.229430e+03,1.427180e+03,3.109950e+03,1.645840e+03,1.435150e+03,1.100520e+03
75%,10.000000,112.000000,83.000000,5.050000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1576.650000,9.998100e+02,6.762180e+03,3.518610e+03,1.798950e+03,2.083290e+03,5.102270e+03,2.629330e+03,2.098440e+03,1.611450e+03
max,49.000000,280.000000,97.000000,1.185000e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,417372.390000,1.152970e+07,1.152890e+07,1.152791e+07,1.152979e+07,1.152979e+07,1.152979e+07,1.152979e+07,1.152979e+07,1.153099e+07


In [16]:
ss = StandardScaler()
ss.fit(X_train)
X_train_ss = ss.transform(X_train)
X_valid_ss = ss.transform(X_valid)
#X_test = ss.transform(X_test)

## Baseline Linear Regression

In [32]:
reg = LinearRegression().fit(X_train_ss, y_train)

In [33]:
y_pred = reg.predict(X_valid_ss)

In [35]:
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 51512.321212


## XGBoost

In [18]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [19]:
xg_reg.fit(X_train_ss,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [37]:
y_pred = xg_reg.predict(X_valid_ss)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 186567.992546


In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [41]:
params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

print((cv_results["test-rmse-mean"]).tail(1))

49    46851.627604
Name: test-rmse-mean, dtype: float64


In [40]:
params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.05,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

print((cv_results["test-rmse-mean"]).tail(1))

49    73742.419271
Name: test-rmse-mean, dtype: float64


In [42]:
params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.2,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

print((cv_results["test-rmse-mean"]).tail(1))

49    37419.154948
Name: test-rmse-mean, dtype: float64


In [44]:
## Hyper Parameter Optimization
regressor=xgboost.XGBRegressor()
n_estimators = [100, 500, 1000]
max_depth = [2, 3, 5]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.20]
base_score=[0.25,0.5,0.75]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'booster':booster,
    'base_score':base_score
    }

# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=30,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [45]:
random_cv.fit(X_train,y_train)
end_time = time.perf_counter()

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  4.1min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed: 130.5min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed: 418.6min finished


Boosting took 27392.7047 seconds


In [46]:
random_cv.best_estimator_

XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=5,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [50]:
xg_reg=xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=5, min_child_weight=5, missing=None, n_estimators=1000,
       n_jobs=0, num_parallel_tree=1, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, tree_method='exact', validate_parameters=1,
       verbosity=3, subsample=1)

In [51]:
xg_reg.fit(X_train_ss,y_train)

[09:10:00] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 56 extra nodes, 0 pruned nodes, max_depth=5
[09:10:01] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:10:02] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:10:02] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:10:03] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:10:04] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned

[09:10:24] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:10:25] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 52 extra nodes, 0 pruned nodes, max_depth=5
[09:10:25] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:10:26] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[09:10:26] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:10:27] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 56 extra nodes, 0 pruned

[09:10:48] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=5
[09:10:48] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=5
[09:10:49] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=5
[09:10:49] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:10:50] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 32 extra nodes, 0 pruned nodes, max_depth=5
[09:10:50] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 48 extra nodes, 0 pruned

[09:11:11] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[09:11:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:11:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[09:11:13] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 52 extra nodes, 0 pruned nodes, max_depth=5
[09:11:13] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=5
[09:11:14] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned

[09:11:35] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 56 extra nodes, 0 pruned nodes, max_depth=5
[09:11:36] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=5
[09:11:36] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=5
[09:11:37] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=5
[09:11:37] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[09:11:38] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 30 extra nodes, 0 pruned

...

[09:23:41] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=5
[09:23:43] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 58 extra nodes, 0 pruned nodes, max_depth=5
[09:23:46] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=5
[09:23:48] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:23:50] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[09:23:54] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 40 extra nodes, 0 pruned

[09:24:54] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 34 extra nodes, 0 pruned nodes, max_depth=5
[09:24:55] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:24:57] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=5
[09:24:59] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 52 extra nodes, 0 pruned nodes, max_depth=5
[09:25:01] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 48 extra nodes, 0 pruned nodes, max_depth=5
[09:25:02] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 50 extra nodes, 0 pruned

[09:26:28] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 52 extra nodes, 0 pruned nodes, max_depth=5
[09:26:31] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=5
[09:26:32] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=5
[09:26:34] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:26:37] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[09:26:39] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 38 extra nodes, 0 pruned

[09:27:37] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[09:27:38] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=5
[09:27:41] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=5
[09:27:42] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 42 extra nodes, 0 pruned nodes, max_depth=5
[09:27:43] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=5
[09:27:45] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\tree\updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned

XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=5,
             min_child_weight=5, missing=None, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=3)

In [53]:
y_pred = xg_reg.predict(X_valid_ss)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 25322.437565


## Keras

In [5]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [8]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))

pipeline = Pipeline(estimators)

kfold = KFold(n_splits=10)

results = cross_val_score(pipeline, X, Y, cv=kfold)

print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py", line 66, in _method_wrapper
    return method(self, *args, **kwargs)
  File

C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py", line 66, in _method_wrapper
    return method(self, *args, **kwargs)
  File

C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py", line 66, in _method_wrapper
    return method(self, *args, **kwargs)
  File

C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py", line 66, in _method_wrapper
    return method(self, *args, **kwargs)
  File

C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py", line 66, in _method_wrapper
    return method(self, *args, **kwargs)
  File

Standardized: nan (nan) MSE


C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\wtbha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py", line 66, in _method_wrapper
    return method(self, *args, **kwargs)
  File